# RanCh (Integrative Analysis of RNA-Seq and ChIP-Seq Data)

Tools

    1. DESEQ2 - Differential Gene Expression Analysis
    2. BETA - Integrative Analysis of RNA-Seq and ChIP-Seq Data

## RNA-Seq

In [1]:
# Imports
library(DESeq2)
library(dplyr)
library(stringr)
library(tidyr)

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loading required package: SummarizedExperiment

Loading required package: M

In [4]:
# Upload Feature Count Matrix and Metadata
countData <- read.csv("Analysis/E-GEOD-47399-raw-counts.tsv", sep="\t", header=TRUE, row.names=1)
metaData <- read.csv("Analysis/E-GEOD-47399-experiment-design.tsv", sep="\t", header=TRUE, row.names=1)
metaData <- metaData[match(colnames(countData), row.names(metaData)), ]

In [6]:
colnames(metaData)

[1] "Sample.Characteristic.RNA.interference."              
 [2] "Sample.Characteristic.Ontology.Term.RNA.interference."
 [3] "Sample.Characteristic.cell.line."                     
 [4] "Sample.Characteristic.Ontology.Term.cell.line."       
 [5] "Sample.Characteristic.organism."                      
 [6] "Sample.Characteristic.Ontology.Term.organism."        
 [7] "Sample.Characteristic.passage."                       
 [8] "Sample.Characteristic.Ontology.Term.passage."         
 [9] "Sample.Characteristic.phenotype."                     
[10] "Sample.Characteristic.Ontology.Term.phenotype."       
[11] "Factor.Value.phenotype."                              
[12] "Factor.Value.Ontology.Term.phenotype."                
[13] "Analysed"

In [10]:
# Create DESEq2 Object
dds <- DESeqDataSetFromMatrix(countData = countData, colData = metaData, design = ~Factor.Value.phenotype.)
dds <- DESeq(dds)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
 

In [11]:
res <- results(dds)
resdf <- as.data.frame(res)
colnames(resdf) <- c("baseMean", "log2FoldChange", "lfcSE", "stat", "pvalue", "padj")
resdf <- resdf[c("log2FoldChange", "pvalue")]

In [29]:
# Remove the rows with NA values
resdf <- na.omit(resdf)

# Filter the results based on the pvalue and log2FoldChange
resdf <- resdf[which(resdf$pvalue < 0.05), ]
resdf <- resdf[which(abs(resdf$log2FoldChange) > 1), ]

# Write the results to a file
write.table(resdf, file = paste0('Analysis/BETA_Demo/results.csv'), sep = '\t',col.names = FALSE)

## Integration with ChIP-Seq

In [39]:
system('/home/shrey/miniconda3/envs/beta/bin/BETA basic -p Analysis/MAX_HeLa_S3_chip_seq.bed -e Analysis/BETA_Demo/results.csv -g hg38 -k BSF -n demo -o Analysis/BETA_Demo --gname2 -c 0.05', intern = TRUE)

Warning message in system("/home/shrey/miniconda3/envs/beta/bin/BETA basic -p Analysis/MAX_HeLa_S3_chip_seq.bed -e Analysis/BETA_Demo/results.csv -g hg38 -k BSF -n demo -o Analysis/BETA_Demo --gname2 -c 0.05", :
“running command '/home/shrey/miniconda3/envs/beta/bin/BETA basic -p Analysis/MAX_HeLa_S3_chip_seq.bed -e Analysis/BETA_Demo/results.csv -g hg38 -k BSF -n demo -o Analysis/BETA_Demo --gname2 -c 0.05' had status 1”


[1] "[01:57:14] Argument List: "                                                                                                  
 [2] "[01:57:14] Name = demo"                                                                                                      
 [3] "[01:57:14] Peak File = Analysis/MAX_HeLa_S3_chip_seq.bed"                                                                    
 [4] "[01:57:14] Top Peaks Number = 10000"                                                                                         
 [5] "[01:57:14] Distance = 100000 bp"                                                                                             
 [6] "[01:57:14] Genome = hg38"                                                                                                    
 [7] "[01:57:14] Expression File = Analysis/BETA_Demo/results.csv"                                                                 
 [8] "[01:57:14] BETA specific Expression Type"                                                                                    
 [9] "[01:57:14] Number of differential expressed genes = 0.5"                                                                     
[10] "[01:57:14] Differential expressed gene FDR Threshold = 1"                                                                    
[11] "[01:57:14] Up/Down Prediction Cutoff = 0.050000"                                                                             
[12] "[01:57:14] Function prediction based on regulatory potential"                                                                
[13] "[01:57:14] Check Analysis/MAX_HeLa_S3_chip_seq.bed successfully!"                                                            
[14] "[01:57:14] \"CD38\"\t-2.05480357219086\t0.00101462218877279"                                                                 
[15] " is not the header of the expression file"                                                                                   
[16] "[01:57:14] Checking the differential expression infomation..."                                                               
[17] "[01:57:14] Take the first line with Differential Information as an example: \"CD38\"\t-2.05480357219086\t0.00101462218877279"
[18] ""                                                                                                                            
[19] "[01:57:14] Differential Expression file format successful passed"                                                            
[20] "[01:57:14] You do not like filter peak by CFCT boundary, it will be filtered only by the distance"                           
[21] "[01:57:14] Read file <Analysis/MAX_HeLa_S3_chip_seq.bed> OK! All <10000> peaks."                                             
[22] "[01:57:19] Process <51276> genes"                                                                                            
[23] "[01:57:28] Finished! Preliminary results saved into temporary file: <demo.txt>"                                              
[24] "[238, 508]"                                                                                                                  
[25] "[01:57:28] Genes were seprated to two parts: up regulated and down regulated."                                               
[26] "[01:57:28] Prepare file for the Up/Down Test"                                                                                
[27] "[01:57:28] Finished, Find the result in demo_function_prediction.pdf"                                                        
attr(,"status")
[1] 1
attr(,"errmsg")
[1] "Resource temporarily unavailable"